Progetto per trovare parole adatte ad un cruciverba

Prima dobbiamo importare il file .txt con il nostro dizionario di tutte le parole. una parola per riga; e inserirle in un vettore che consenta di iterare lettera per lettera quando dovremo cercare corrispondenze

In [75]:
import os
import glob

def carica_tutti_dizionari(cartella_dizionari="dizionari"):
    """
    Carica automaticamente tutti i file .txt dalla cartella dizionari.
    
    Args:
        cartella_dizionari: nome della cartella contenente i dizionari (default: "dizionari")
    
    Returns:
        Dizionario con nome_file -> lista di parole
    """
    dizionari = {}
    
    # Se la cartella non esiste, cerca i file .txt nella cartella corrente
    if not os.path.exists(cartella_dizionari):
        print(f"Cartella '{cartella_dizionari}' non trovata, cerco file .txt nella cartella corrente...")
        pattern_file = "*.txt"
        base_path = ""
    else:
        pattern_file = os.path.join(cartella_dizionari, "*.txt")
        base_path = cartella_dizionari + "/"
    
    # Trova tutti i file .txt
    file_dizionari = glob.glob(pattern_file)
    
    if not file_dizionari:
        print(f"Nessun file .txt trovato in '{cartella_dizionari if os.path.exists(cartella_dizionari) else 'cartella corrente'}'")
        return dizionari
    
    print(f"Trovati {len(file_dizionari)} file dizionario:")
    
    for percorso_file in file_dizionari:
        nome_file = os.path.basename(percorso_file).replace('.txt', '')
        try:
            with open(percorso_file, 'r', encoding='utf-8') as f:
                parole = [list(riga.strip().lower()) for riga in f if riga.strip()]
                # Rimuovi duplicati
                parole_uniche = []
                parole_viste = set()
                for parola in parole:
                    parola_str = ''.join(parola)
                    if parola_str not in parole_viste:
                        parole_uniche.append(parola)
                        parole_viste.add(parola_str)
                
                dizionari[nome_file] = parole_uniche
                print(f"  - {nome_file}: {len(parole_uniche)} parole")
        
        except Exception as e:
            print(f"  - Errore caricando {nome_file}: {e}")
    
    return dizionari

def importa_dizionario(percorsi_file):
    """
    Funzione legacy per compatibilità con il codice esistente.
    """
    # Se è una stringa, convertila in lista
    if isinstance(percorsi_file, str):
        percorsi_file = [percorsi_file]
    
    parole = []
    for percorso_file in percorsi_file:
        try:
            with open(percorso_file, 'r', encoding='utf-8') as f:
                parole_file = [list(riga.strip().lower()) for riga in f if riga.strip()]
                parole.extend(parole_file)
                print(f"Importate {len(parole_file)} parole da {percorso_file}")
        except FileNotFoundError:
            print(f"Attenzione: File {percorso_file} non trovato")
        except Exception as e:
            print(f"Errore durante l'importazione di {percorso_file}: {e}")
    
    # Rimuovi duplicati mantenendo l'ordine
    parole_uniche = []
    parole_viste = set()
    for parola in parole:
        parola_str = ''.join(parola)
        if parola_str not in parole_viste:
            parole_uniche.append(parola)
            parole_viste.add(parola_str)
    
    print(f"Totale parole uniche importate: {len(parole_uniche)}")
    return parole_uniche

Adesso dobbiamo creare la funzione per la ricerca mediante caratteri jolly

In [89]:
import re

def cerca_parole(parole, pattern):
    """
    Cerca parole che corrispondono al pattern con wildcard avanzate.
    La ricerca è case insensitive.
    
    Wildcard supportate:
    - '*': qualsiasi carattere
    - '@': vocale (a, e, i, o, u)
    - '#': consonante (qualsiasi lettera che non sia vocale)
    
    Esempi:
    - '**o**': parole di 5 lettere con 'o' come terza lettera
    - 'c@sa': parole di 4 lettere che iniziano con 'c', hanno una vocale, poi 's', poi 'a'
    - '#@ro': parole di 4 lettere: consonante, vocale, 'r', 'o'
    """
    # Converte il pattern in minuscolo per confronto case insensitive
    pattern = pattern.lower()
    
    # Definisce vocali e consonanti
    vocali = set('aeiou')
    consonanti = set('bcdfghjklmnpqrstvwxyz')
    
    risultati = []
    for parola in parole:
        if len(parola) != len(pattern):
            continue
        corrisponde = True
        for p_char, pat_char in zip(parola, pattern):
            if pat_char == '*':
                # Qualsiasi carattere va bene
                continue
            elif pat_char == '@':
                # Deve essere una vocale
                if p_char not in vocali:
                    corrisponde = False
                    break
            elif pat_char == '#':
                # Deve essere una consonante
                if p_char not in consonanti:
                    corrisponde = False
                    break
            else:
                # Carattere specifico
                if p_char != pat_char:
                    corrisponde = False
                    break
        if corrisponde:
            risultati.append(''.join(parola))
    return risultati

def cerca_sottostringa(parole, sottostringa, filtro_lunghezza=None):
    """
    Cerca tutte le parole che contengono la sottostringa specificata,
    indipendentemente dalla lunghezza della parola.
    
    Args:
        parole: Lista di parole dove ogni parola è una lista di caratteri
        sottostringa: Stringa da cercare dentro le parole
        filtro_lunghezza: Filtro per la lunghezza delle parole (es: "=7", "<5", ">10")
    
    Returns:
        Lista di parole che contengono la sottostringa
    
    Esempi:
    - cerca_sottostringa(parole, 'tro') → trova 'metro', 'altro', 'contro', ecc.
    - cerca_sottostringa(parole, 'tro', '=5') → trova solo parole di 5 lettere con 'tro'
    - cerca_sottostringa(parole, 'zione', '>8') → trova parole con 'zione' e più di 8 lettere
    """
    sottostringa = sottostringa.lower()
    risultati = []
    
    for parola in parole:
        parola_str = ''.join(parola)
        if sottostringa in parola_str:
            # Applica filtro lunghezza se specificato
            if filtro_lunghezza:
                lunghezza = len(parola_str)
                if not verifica_lunghezza(lunghezza, filtro_lunghezza):
                    continue
            risultati.append(parola_str)
    
    return risultati

def verifica_lunghezza(lunghezza, filtro):
    """
    Verifica se la lunghezza della parola soddisfa il filtro.
    
    Args:
        lunghezza: Lunghezza della parola
        filtro: Stringa di filtro (es: "=7", "<5", ">10")
    
    Returns:
        True se la lunghezza soddisfa il filtro, False altrimenti
    """
    if not filtro:
        return True
    
    if filtro.startswith('='):
        target = int(filtro[1:])
        return lunghezza == target
    elif filtro.startswith('<'):
        target = int(filtro[1:])
        return lunghezza < target
    elif filtro.startswith('>'):
        target = int(filtro[1:])
        return lunghezza > target
    
    return True

def cerca_in_tutti_dizionari(dizionari, pattern, mostra_tutti=True, max_risultati_per_dizionario=None):
    """
    Cerca un pattern in tutti i dizionari e mostra i risultati divisi per dizionario.
    
    Args:
        dizionari: Dizionario con nome_dizionario -> lista_parole
        pattern: Pattern di ricerca con wildcard OPPURE sottostringa se usa -
        mostra_tutti: Se True mostra tutti i risultati, altrimenti limita
        max_risultati_per_dizionario: Numero massimo di risultati da mostrare per ogni dizionario (solo se mostra_tutti=False)
    
    Returns:
        Dizionario con risultati per ogni dizionario
        
    Nota: Se il pattern inizia con '-', verrà trattato come ricerca di sottostringa.
    Esempi: 
    - '-tro' cerca tutte le parole che contengono 'tro'
    - '-tro=5' cerca parole che contengono 'tro' di esattamente 5 lettere
    - '-tro<7' cerca parole che contengono 'tro' di meno di 7 lettere
    - '-tro>10' cerca parole che contengono 'tro' di più di 10 lettere
    """
    tutti_risultati = {}
    totale_complessivo = 0
    
    # Determina il tipo di ricerca
    if pattern.startswith('-'):
        tipo_ricerca = "sottostringa"
        termine_ricerca = pattern[1:]  # Rimuove il -
        
        # Estrai filtro lunghezza se presente
        filtro_lunghezza = None
        for op in ['=', '<', '>']:
            if op in termine_ricerca:
                parti = termine_ricerca.split(op)
                if len(parti) == 2:
                    termine_ricerca = parti[0]
                    try:
                        filtro_lunghezza = op + parti[1]
                    except:
                        pass
                break
        
        if filtro_lunghezza:
            print(f"🔍 Ricerca sottostringa: '{termine_ricerca}' con filtro lunghezza '{filtro_lunghezza}'")
        else:
            print(f"🔍 Ricerca sottostringa: '{termine_ricerca}' (parole che contengono questa sequenza)")
    else:
        tipo_ricerca = "pattern"
        termine_ricerca = pattern
        filtro_lunghezza = None
        print(f"🔍 Ricerca pattern: '{pattern}' (lunghezza fissa con wildcard)")
    
    print("=" * 60)
    
    for nome_dizionario, parole in dizionari.items():
        if tipo_ricerca == "sottostringa":
            risultati = cerca_sottostringa(parole, termine_ricerca, filtro_lunghezza)
        else:
            risultati = cerca_parole(parole, pattern)
            
        tutti_risultati[nome_dizionario] = risultati
        totale_complessivo += len(risultati)
        
        print(f"\n📚 {nome_dizionario.upper()}: {len(risultati)} risultati")
        if risultati:
            if mostra_tutti or max_risultati_per_dizionario is None:
                # Mostra tutti i risultati
                for i, parola in enumerate(risultati, 1):
                    print(f"  {i:3d}. {parola}")
            else:
                # Mostra solo i primi risultati
                risultati_da_mostrare = risultati[:max_risultati_per_dizionario]
                for i, parola in enumerate(risultati_da_mostrare, 1):
                    print(f"  {i:2d}. {parola}")
                
                if len(risultati) > max_risultati_per_dizionario:
                    print(f"     ... e altri {len(risultati) - max_risultati_per_dizionario} risultati")
        else:
            print("     Nessun risultato trovato")
    
    print("=" * 60)
    print(f"📊 TOTALE: {totale_complessivo} parole trovate in {len(dizionari)} dizionari")
    
    return tutti_risultati

In [90]:
# Test dei filtri di lunghezza per le sottostringhe
print("🧪 TEST: Filtri di lunghezza per le sottostringhe")
print("=" * 50)

# Test 1: Cerca parole che contengono 'tro' di esattamente 5 lettere
print("\n1️⃣ Ricerca: parole con 'tro' di ESATTAMENTE 5 lettere")
cerca_in_tutti_dizionari(dizionari, "-tro=5", mostra_tutti=True)

print("\n" + "="*80 + "\n")

# Test 2: Cerca parole che contengono 'zione' di più di 8 lettere
print("2️⃣ Ricerca: parole con 'zione' di PIÙ DI 8 lettere")  
cerca_in_tutti_dizionari(dizionari, "-zione>8", mostra_tutti=True)

print("\n" + "="*80 + "\n")

# Test 3: Cerca parole che contengono 'mente' di meno di 10 lettere
print("3️⃣ Ricerca: parole con 'mente' di MENO DI 10 lettere")
cerca_in_tutti_dizionari(dizionari, "-mente<10", mostra_tutti=True)

🧪 TEST: Filtri di lunghezza per le sottostringhe

1️⃣ Ricerca: parole con 'tro' di ESATTAMENTE 5 lettere
🔍 Ricerca sottostringa: 'tro' con filtro lunghezza '=5'

📚 ENG: 53 risultati
    1. altro
    2. atro-
    3. etrog
    4. intro
    5. jitro
    6. latro
    7. metro
    8. nitro
    9. petro
   10. retro
   11. stroh
   12. stroy
   13. strom
   14. strop
   15. strow
   16. troad
   17. troak
   18. troas
   19. troat
   20. troca
   21. troch
   22. trock
   23. troco
   24. trode
   25. trodi
   26. troff
   27. troft
   28. trogs
   29. troic
   30. trois
   31. troys
   32. troke
   33. troll
   34. tromp
   35. trona
   36. tronc
   37. trone
   38. tronk
   39. troop
   40. troot
   41. trooz
   42. trop-
   43. trope
   44. tropy
   45. troth
   46. trots
   47. troue
   48. troup
   49. trout
   50. trouv
   51. trove
   52. trows
   53. vitro

📚 ITA: 34 risultati
    1. altro
    2. antro
    3. astro
    4. botro
    5. entro
    6. estro
    7. iatro
    8. intro
    

{'eng': ['agilmente',
  'alimented',
  'alimenter',
  'augmented',
  'augmenter',
  'cemented',
  'cementer',
  'cementers',
  'clemente',
  'commented',
  'commenter',
  'demented',
  'fermented',
  'fermenter',
  'fomented',
  'fomenter',
  'fomenters',
  'furmente',
  'garmented',
  'lamented',
  'lamenter',
  'lamenters',
  'mentery',
  'mentes',
  'pigmented',
  'pimentel',
  'raimented',
  'segmented',
  'segmenter',
  'sementera',
  'tormented',
  'tormenter',
  'tourmente',
  'vivamente'],
 'ita': ['abilmente',
  'acremente',
  'agilmente',
  'altamente',
  'aumentera',
  'aumentero',
  'caramente',
  'cimentera',
  'cimentero',
  'clemente',
  'cupamente',
  'demente',
  'dirimente',
  'dormente',
  'duramente',
  'equamente',
  'esimente',
  'fomentera',
  'fremente',
  'gaiamente',
  'gemente',
  'giumente',
  'lamentela',
  'lamentele',
  'lamentera',
  'lamentero',
  'lealmente',
  'malamente',
  'mente',
  'mentendo',
  'meramente',
  'mutamente',
  'oralmente',
  'parime

Adesso il programma può essere utilizzato per cercare parole in un dizionario.

In [77]:
# Carica automaticamente tutti i dizionari disponibili

# Carica tutti i dizionari dalla cartella "dizionari" (o dalla cartella corrente se non esiste)
dizionari = carica_tutti_dizionari()

print(f"\n✅ Sistema pronto! Caricate {sum(len(parole) for parole in dizionari.values())} parole totali")
print(f"📁 Dizionari disponibili: {', '.join(dizionari.keys())}")
print("\nOra puoi usare:")
print("- cerca_in_tutti_dizionari(dizionari, 'pattern') per cercare in tutti")
print("- cerca_parole(dizionari['nome'], 'pattern') per cercare in uno specifico")

Trovati 2 file dizionario:
  - eng: 466546 parole
  - eng: 466546 parole
  - ita: 279894 parole

✅ Sistema pronto! Caricate 746440 parole totali
📁 Dizionari disponibili: eng, ita

Ora puoi usare:
- cerca_in_tutti_dizionari(dizionari, 'pattern') per cercare in tutti
- cerca_parole(dizionari['nome'], 'pattern') per cercare in uno specifico
  - ita: 279894 parole

✅ Sistema pronto! Caricate 746440 parole totali
📁 Dizionari disponibili: eng, ita

Ora puoi usare:
- cerca_in_tutti_dizionari(dizionari, 'pattern') per cercare in tutti
- cerca_parole(dizionari['nome'], 'pattern') per cercare in uno specifico


In [78]:
# Esempi con il nuovo sistema di ricerca automatica
# Esegui prima la cella 7 per caricare i dizionari

# Esempio 1: Ricerca pattern tradizionale (lunghezza fissa)
print("🎯 RICERCA PATTERN (lunghezza fissa)")
risultati_pattern = cerca_in_tutti_dizionari(dizionari, '#@ro')

print("\n" + "="*70)

# Esempio 2: Ricerca sottostringa (qualsiasi lunghezza) - NOVITÀ!
print("\n🌟 RICERCA SOTTOSTRINGA (qualsiasi lunghezza) 🌟")
risultati_sottostringa = cerca_in_tutti_dizionari(dizionari, '-tro')

print("\n" + "="*70)

# Esempio 3: Altra ricerca sottostringa
print("\n🔥 ALTRA RICERCA SOTTOSTRINGA 🔥")
risultati_zione = cerca_in_tutti_dizionari(dizionari, '-zione')

🎯 RICERCA PATTERN (lunghezza fissa)
🔍 Ricerca pattern: '#@ro' (lunghezza fissa con wildcard)

📚 ENG: 30 risultati
    1. biro
    2. boro
    3. buro
    4. caro
    5. cero
    6. ciro
    7. coro
    8. dero
    9. doro
   10. duro
   11. faro
   12. garo
   13. giro
   14. hero
   15. hiro
   16. karo
   17. kero
   18. loro
   19. maro
   20. mero
   21. miro
   22. moro
   23. nero
   24. pero
   25. piro
   26. sero
   27. taro
   28. tiro
   29. toro
   30. zero

📚 ITA: 34 risultati
    1. baro
    2. biro
    3. boro
    4. caro
    5. cero
    6. ciro
    7. coro
    8. curo
    9. daro
   10. diro
   11. duro
   12. faro
   13. foro
   14. giro
   15. loro
   16. maro
   17. mero
   18. miro
   19. moro
   20. muro
   21. nero
   22. paro
   23. pero
   24. poro
   25. puro
   26. raro
   27. saro
   28. tiro
   29. toro
   30. turo
   31. varo
   32. vero
   33. viro
   34. zero
📊 TOTALE: 64 parole trovate in 2 dizionari


🌟 RICERCA SOTTOSTRINGA (qualsiasi lunghezza) 🌟
🔍 Ric

## 🎯 Guida al Sistema di Ricerca Avanzata

### 📁 Caricamento Automatico
Il programma carica automaticamente **tutti i file .txt** trovati in:
1. Cartella `dizionari/` (se esiste)
2. Cartella corrente (se `dizionari/` non esiste)

### 🔍 Due Tipi di Ricerca:

#### 1. **Ricerca Pattern (lunghezza fissa)**
- **`*`** = qualsiasi carattere
- **`@`** = vocale (a, e, i, o, u)
- **`#`** = consonante (tutte le altre lettere)

#### 2. **Ricerca Sottostringa (qualsiasi lunghezza)**
- **`-testo`** = cerca tutte le parole che contengono 'testo'
- Esempi: `-tro`, `-zione`, `-casa`

### 📝 Esempi di Pattern:
- `c*sa` = parole di 4 lettere: c-qualsiasi-s-a
- `#@sa` = parole di 4 lettere: consonante-vocale-s-a
- `*@*@*` = parole di 5 lettere con vocali in 2ª e 4ª posizione
- `#@#` = parole di 3 lettere: consonante-vocale-consonante

### 🔍 Esempi di Sottostringa:
- `-tro` = trova 'metro', 'altro', 'contro', 'elettronico', ecc.
- `-zione` = trova 'azione', 'nazione', 'stazione', 'attenzione', ecc.
- `-casa` = trova 'casa', 'casale', 'casanova', 'ricascare', ecc.
- `-ing` = trova parole inglesi con 'ing' (se hai dizionario inglese)

### 🚀 Funzioni Principali:
- `cerca_in_tutti_dizionari(dizionari, 'pattern')` → cerca pattern o sottostringa in tutti
- `cerca_parole(dizionari['nome'], 'pattern')` → cerca pattern in uno specifico
- `cerca_sottostringa(dizionari['nome'], 'testo')` → cerca sottostringa in uno specifico

### 📊 Output Completo:
Per default **TUTTI i risultati** sono mostrati! 
- Usa `mostra_tutti=False` per limitare l'output
- Perfetto per trovare tutte le opzioni disponibili per cruciverba

In [88]:
# 🎯 RICERCA INTERATTIVA AVANZATA 🎯
# Cerca il tuo pattern in tutti i dizionari disponibili

print("Tipi di ricerca disponibili:")
print("📏 Pattern (lunghezza fissa): usa *, @, # - esempio: '#@ro'")
print("🔍 Sottostringa (qualsiasi lunghezza): inizia con - - esempio: '-tro'")
print()

pattern_ricerca = input("🔍 Inserisci la ricerca (pattern o -sottostringa): ")

if pattern_ricerca:
    print("\n" + "🎯" * 25)
    risultati_finali = cerca_in_tutti_dizionari(dizionari, pattern_ricerca)
    
    # Mostra suggerimenti basati sul tipo di ricerca
    if pattern_ricerca.startswith('-'):
        print(f"\n💡 Hai cercato la sottostringa '{pattern_ricerca[1:]}' in parole di qualsiasi lunghezza")
        print("💡 Prova anche: pattern con lunghezza fissa usando *, @, #")
    else:
        print(f"\n💡 Hai cercato un pattern di lunghezza fissa: {len(pattern_ricerca)} lettere")
        print(f"💡 Prova anche: '-{pattern_ricerca.replace('*', '').replace('@', '').replace('#', '')}' per ricerca libera")
    
    print(f"\n🔧 Per vedere solo risultati limitati, usa:")
    print(f"   cerca_in_tutti_dizionari(dizionari, '{pattern_ricerca}', mostra_tutti=False, max_risultati_per_dizionario=10)")
else:
    print("❌ Ricerca vuota, riprova!")

Tipi di ricerca disponibili:
📏 Pattern (lunghezza fissa): usa *, @, # - esempio: '#@ro'
🔍 Sottostringa (qualsiasi lunghezza): inizia con - - esempio: '-tro'


🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
🔍 Ricerca pattern: '***se*' (lunghezza fissa con wildcard)

📚 ENG: 624 risultati
    1. abased
    2. abaser
    3. abases
    4. abused
    5. abusee
    6. abuser
    7. abuses
    8. alisen
    9. amused
   10. amusee
   11. amuser
   12. amuses
   13. anises
   14. anuses
   15. arised
   16. arisen
   17. ariser
   18. arises
   19. avoset
   20. axised
   21. axises
   22. baases
   23. basses
   24. basset
   25. beisel
   26. belsen
   27. bensel
   28. bensen
   29. bessel
   30. besses
   31. betsey
   32. biased
   33. biases
   34. birses
   35. bisset
   36. bitser
   37. bonser
   38. bossed
   39. bosser
   40. bosses
   41. bosset
   42. boused
   43. bouser
   44. bouses
   45. bowsed
   46. bowser
   47. bowses
   48. brasen
   49. bresee
   50. brises
   51. broses
   52. bunsen
   53.